## Image Captionning

### Imports

In [16]:
import torch, sys, torchvision
from PIL import Image
from dataloader.get import DataGetter
sys.path.append('./LAVIS/')
from lavis.models import load_model_and_preprocess

In [17]:
dg=DataGetter()
print(torch.cuda.is_available())

True


In [18]:
# setup device to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")

# we associate a model with its preprocessors to make it easier for inference.
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)
# uncomment to use base model
# model, vis_processors, _ = load_model_and_preprocess(
#     name="blip_caption", model_type="base_coco", is_eval=True, device=device
# )
vis_processors.keys()

def get_caption(image,device):
    
    raw_image = Image.fromarray(image)
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    
    # we'll first use beam search to generate a caption
    caption = model.generate({"image": image})
    
    # we'll then use nucleus sampling to get multiple captions
    # and more precise captions (you can uncomment the line below)
    
    # caption = model.generate({"image": image}, use_nucleus_sampling=True, num_captions=3)
    
    # store the generated caption in a variable
    return caption


    

In [43]:
import matplotlib.pyplot as plt

def caption_from_image_file(x):
    return [get_caption(i,device) for i in x.load()]

df = dg.getData("train")

df_test = df.head(5)


# df_test['captions'] = df_test.images.apply(caption_from_image_file)

# df_test.to_csv('test.csv',index=False)

# # free up cuda memory
# torch.cuda.empty_cache()

# convert all columns that are objects to string
df_test = df_test.astype({col: 'str' for col in df_test.columns[df_test.dtypes == 'object']})





from pyspark.sql import SparkSession
import os

os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-19"
os.environ["SPARK_HOME"] = "C:/Program Files/Spark/spark-3.3.1-bin-hadoop3"




# Create a Spark session
spark = SparkSession.builder.appName("ParallelCaptioning").getOrCreate()

# Convert the Pandas dataframe to a Spark dataframe
df_spark = spark.createDataFrame(df_test)

# Use the map transformation to apply the function to each row
df_captioned = df_spark.rdd.map(lambda row: (row[0], row[1], caption_from_image_file(row[1]))).toDF(['filename', 'images', 'captions'])

# # # Convert the Spark dataframe back to a Pandas dataframe
# # df_captioned = df_captioned.toPandas()

# # # Write the data to a CSV file
# # df_captioned.to_csv('test_parallel.csv', index=False)

# # Stop the Spark session
# spark.stop()

c:\Users\elmah\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\elmah\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
Traceback (most recent call last):
  File "c:\Users\elmah\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "c:\Users\elmah\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "c:\Users\elmah\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 602,

PicklingError: Could not serialize object: TypeError: cannot pickle 'weakref' object